## Converted Text File

In [ ]:
import xml.etree.ElementTree as ET
from collections import defaultdict
import os

def parse_pnml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    transitions = {}
    places = {}
    arcs = defaultdict(list)
    place_to_transition = defaultdict(list)
    transition_to_place = defaultdict(list)
    base_transition_map = defaultdict(list)

    transition_names = {}

    for transition in root.findall(".//transition"):
        trans_id = transition.get("id")

        name_elem = transition.find(".//name")
        if name_elem is not None:
            text_elem = name_elem.find(".//text")
            if text_elem is not None and text_elem.text:
                name = text_elem.text.strip()
            else:
                name = trans_id
        else:
            name = trans_id

        base_name = name
        if name in transition_names:
            suffix = trans_id[-3:]
            name = f"{name}_{suffix}"

        transition_names[name] = trans_id
        transitions[trans_id] = name
        base_transition_map[base_name].append(trans_id)

    # Collect role names from transitionResource
    transition_roles = {}
    for transition in root.findall(".//transition"):
        trans_id = transition.get("id")
        res_elem = transition.find(".//transitionResource")
        if res_elem is not None:
            role = res_elem.attrib.get("roleName")
            if role:
                transition_roles[trans_id] = role

    # Add role names to transition labels
    for trans_id in transitions:
        role = transition_roles.get(trans_id)
        if role:
            transitions[trans_id] += f" [rolename: {role}]"

    for place in root.findall(".//place"):
        place_id = place.get("id")

        name_elem = place.find(".//name")
        if name_elem is not None:
            text_elem = name_elem.find(".//text")
            if text_elem is not None and text_elem.text:
                place_name = text_elem.text.strip()
            else:
                place_name = place_id
        else:
            place_name = place_id

        places[place_id] = place_name

    for arc in root.findall(".//arc"):
        source = arc.get("source")
        target = arc.get("target")

        arcs[source].append(target)

        if source in places and target in transitions:
            place_to_transition[source].append(target)
        elif source in transitions and target in places:
            transition_to_place[source].append(target)

    return transitions, places, arcs, place_to_transition, transition_to_place, base_transition_map

def find_start_transitions(transitions, places, place_to_transition, transition_to_place):
    start_transitions = []

    for t_id in transitions:
        incoming_places = [
            p for p, t_list in place_to_transition.items()
            if t_id in t_list
        ]
        has_incoming_transition = False

        for p in incoming_places:
            for x, out_places in transition_to_place.items():
                if p in out_places:
                    has_incoming_transition = True
                    break
            if has_incoming_transition:
                break

        if not has_incoming_transition:
            start_transitions.append(t_id)

    return start_transitions

def detect_structure(transitions, places, place_to_transition, transition_to_place, base_transition_map, start_nodes=None):
    structure = []
    visited = set()
    rec_stack = set()

    def get_next_transitions(trans_id):
        next_ts = []
        if trans_id in transition_to_place:
            for pl in transition_to_place[trans_id]:
                next_ts.extend(place_to_transition.get(pl, []))
        return list(set(next_ts))

    def get_incoming_places(trans_id):
        result = []
        for pl, trans_list in place_to_transition.items():
            if trans_id in trans_list:
                result.append(pl)
        return result

    def is_xor_join(trans_id):
        trans_name = transitions[trans_id]
        base_name = trans_name.split(' [')[0].split('_')[0]
        related_trans = base_transition_map.get(base_name, [])
        if len(related_trans) <= 1:
            return False

        incoming_places = get_incoming_places(trans_id)
        if len(incoming_places) <= 1:
            return False

        for place in incoming_places:
            place_targets = place_to_transition[place]
            if not any(t in related_trans for t in place_targets):
                return False

        return True

    def traverse(node, depth=0):
        if node in rec_stack:
            structure.append("    " * depth + f"{transitions[node]} (loop)")
            return

        if node in visited:
            return

        rec_stack.add(node)

        next_nodes = get_next_transitions(node)
        transition_labels = [transitions.get(t, t) for t in next_nodes]

        incoming_places = get_incoming_places(node)
        outgoing_places = transition_to_place.get(node, [])

        if len(incoming_places) > 1:
            if is_xor_join(node):
                place_names = [places.get(pl, pl) for pl in incoming_places]
                structure.append("    " * depth + f"{{ {', '.join(place_names)} }} ->? {transitions[node]} (XOR-join)")
            else:
                place_names = [places.get(pl, pl) for pl in incoming_places]
                structure.append("    " * depth + f"{{ {', '.join(place_names)} }} -> {transitions[node]} (AND-join)")

        if len(outgoing_places) > 1:
            out_trans_set = set()
            for p in outgoing_places:
                out_trans_set.update(place_to_transition.get(p, []))
            structure.append(
                "    " * depth 
                + f"{transitions[node]} ->{{ {', '.join(transitions[t] for t in out_trans_set)} }} (AND-split)"
            )
        elif len(next_nodes) > 1:
            structure.append(
                "    " * depth
                + f"{transitions[node]} ->? {{ {', '.join(transition_labels)} }} (XOR-split)"
            )
        elif len(next_nodes) == 1:
            nxt = next_nodes[0]
            inc_for_nxt = get_incoming_places(nxt)
            if len(inc_for_nxt) > 1:
                pass
            else:
                structure.append("    " * depth + f"{transitions[node]} -> {transitions[nxt]}")
        else:
            structure.append("    " * depth + f"{transitions[node]} -> End")

        for target in next_nodes:
            traverse(target, depth + 1)

        rec_stack.remove(node)
        visited.add(node)

    if not start_nodes:
        discovered_starts = find_start_transitions(transitions, places, place_to_transition, transition_to_place)
        if discovered_starts:
            start_nodes = discovered_starts
        else:
            start_nodes = [next(iter(transitions), None)]

    for s in start_nodes:
        if s is not None:
            traverse(s)
    return "\n".join(structure)

if __name__ == "__main__":
    file_path = r"Insert the file path"
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    output_file = os.path.join(os.path.dirname(file_path), f"{base_name}_output.txt")
    transitions, places, arcs, place_to_transition, transition_to_place, base_transition_map = parse_pnml(file_path)
    start_trans_list = find_start_transitions(transitions, places, place_to_transition, transition_to_place)
    start_trans_output = "Detected start transitions: " + str([transitions[sid] for sid in start_trans_list])
    process_structure = detect_structure(
        transitions,
        places,
        place_to_transition,
        transition_to_place,
        base_transition_map,
        start_nodes=start_trans_list
    )
    full_output = f"{start_trans_output}\n\n[Process Structure]\n{process_structure}"

    # Print to console
    print(full_output)

    # Save to output file with dynamic name
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(full_output)

## Rule1

In [ ]:
import os
from openai import OpenAI


# Get API key from environment variable
client = OpenAI(api_key="Insert API key")





def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def baseline_evaluate_text(text_file_content):
    standard_transitions = [
        "registration manually", "registration online", "receive customer registration", "register request",
        "send credit request to UC", "receive result", "not ok", "ok", "send rejection letter", "End",
        "enbale customer profile update", "enable incoming transactions", "potential fraud using ML",
        "inform customer", "enable full access", "disable customer profile update", "disable incoming transactions",
        "report the case to police", "review the potential fraud", "possible fraud",
        "request external fraud investigation", "2days", "receive result", "send reminder"
    ]

    baseline_prompt = f"""
You are given:
1. A list of 24 standard phrases used to describe a process model.
2. A text file describing transitions in a real process.

### Standard Transition Phrases:
{standard_transitions}

### Text File:
{text_file_content}

### Task:
- Carefully analyze the process described in the text.
- Try to map the text transitions to the 24 standard ones (be flexible, exact match is not required).
- Evaluate whether DetectedStartTransitions array length in text file is more than 1.
- If DetectedStartTransitions array length in text file is not more than 1, Return your assessment as false, print "Please note that the expected solution shall support registrating application online and manual."
- If DetectedStartTransitions array length in text file is more than 1,then evaluate that one of the array component must include manual registration or "[rolename:".
- If one of the array component does not include manual registration, Return your assessment as false, print ""Please check if specific role has been assigned or manual registration."
- If DetectedStartTransitions array length in text file is more than 1,and one of the array component includes manual registration or "[rolename:",Return your assessment as true.
- You should Return one word: `true` or `false`. No need to print thr code.
"""
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "user", "content": baseline_prompt}],
    )

    return response.choices[0].message.content.strip()

if __name__ == "__main__":
    current_dir = os.getcwd()
    txt_filename = r"Insert the file path"

    txt_path = os.path.join(current_dir, txt_filename)

    txt_content = read_file_content(txt_path)

    # Baseline Evaluation
    print("\n⚙️ Running baseline evaluation (no DMN guidance)...\n")
    baseline_result = baseline_evaluate_text(txt_content)

    # Final result
    print("\n✅ Baseline Evaluation Result:\n", baseline_result)


## Rule2

In [ ]:
import os
from openai import OpenAI


# Get API key from environment variable
client = OpenAI(api_key="Insert API key") # Replace with your full API key

def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def baseline_evaluate_text(text_file_content):
    standard_transitions = [
        "registration manually", "registration online", "receive customer registration", "register request",
        "send credit request to UC", "receive result", "not ok", "ok", "send rejection letter", "End",
        "enbale customer profile update", "enable incoming transactions", "potential fraud using ML",
        "inform customer", "enable full access", "disable customer profile update", "disable incoming transactions",
        "report the case to police", "review the potential fraud", "possible fraud",
        "request external fraud investigation", "2days", "receive result", "send reminder"
    ]

    baseline_prompt = f"""
You are given:
1. A list of 24 standard phrases used to describe a process model.
2. A text file describing transitions in a real process.

### Standard Transition Phrases:
{standard_transitions}

### Text File:
{text_file_content}

### Task:
- Carefully analyze the process described in the text.
- Try to map the text transitions to the 24 standard ones (be flexible, exact match is not required).
- For each component of DetectedStartTransitions array, a same transition name must come after each component of DetectedStartTransitions array in ProcessTransitions that comes after 
- This transition name must be "send credit request to uc". 
- If the transition name is "send credit request to uc", return true.
- If the transition name is not "send credit request to uc", return false and print "Please note that the process shall check customer's credit after registration."
- You should Return one word: `true` or `false`. No need to print thr code.
"""
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "user", "content": baseline_prompt}],
    )

    return response.choices[0].message.content.strip()

if __name__ == "__main__":
    current_dir = os.getcwd()
    txt_filename = r"Insert the file path"

    txt_path = os.path.join(current_dir, txt_filename)

    txt_content = read_file_content(txt_path)

    # Baseline Evaluation
    print("\n⚙️ Running baseline evaluation (no DMN guidance)...\n")
    baseline_result = baseline_evaluate_text(txt_content)

    # Final result
    print("\n✅ Baseline Evaluation Result:\n", baseline_result)



## Rule3

In [ ]:
import os
from openai import OpenAI


# Get API key from environment variable
client = OpenAI(api_key="Insert API key") # Replace with your full API key

def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def baseline_evaluate_text(text_file_content):
    standard_transitions = [
        "registration manually", "registration online", "receive customer registration", "register request",
        "send credit request to UC", "receive result", "not ok", "ok", "send rejection letter", "End",
        "enbale customer profile update", "enable incoming transactions", "potential fraud using ML",
        "inform customer", "enable full access", "disable customer profile update", "disable incoming transactions",
        "report the case to police", "review the potential fraud", "possible fraud",
        "request external fraud investigation", "2days", "receive result", "send reminder"
    ]

    baseline_prompt = f"""
You are given:
1. A list of 24 standard phrases used to describe a process model.
2. A text file describing transitions in a real process.

### Standard Transition Phrases:
{standard_transitions}

### Text File:
{text_file_content}

### Task:
- Carefully analyze the process described in the text.
- Try to map the text transitions to the 24 standard ones (be flexible, exact match is not required).
- After transition name "send credit request to uc", it must be to have a transition name "receive result" with the label of "XOR-split".
- If the next transition name is "receive result" with the label of "XOR-split", return true.
- If the transition name is not "receive result" with the label of "XOR-split", return false and print "Please note that the process should behave differently depending on the credit check result."
- You should Return one word: `true` or `false`. No need to print thr code.
"""
    response = client.chat.completions.create(
        model="gpt-4o", 
        messages=[{"role": "user", "content": baseline_prompt}],
    )

    return response.choices[0].message.content.strip()


if __name__ == "__main__":
    current_dir = os.getcwd()
    txt_filename = r"Insert file path"

    txt_path = os.path.join(current_dir, txt_filename)

    txt_content = read_file_content(txt_path)

    # Baseline Evaluation
    print("\n⚙️ Running baseline evaluation (no DMN guidance)...\n")
    baseline_result = baseline_evaluate_text(txt_content)

    # Final result
    print("\n✅ Baseline Evaluation Result:\n", baseline_result)





##  Rule4

In [ ]:
import os
from openai import OpenAI


# Get API key from environment variable
client = OpenAI(api_key="Insert API key") # Replace with your full API key

def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def baseline_evaluate_text(text_file_content):
    standard_transitions = [
        "registration manually", "registration online", "receive customer registration", "register request",
        "send credit request to UC", "receive result", "not ok", "ok", "send rejection letter", "End",
        "enbale customer profile update", "enable incoming transactions", "potential fraud using ML",
        "inform customer", "enable full access", "disable customer profile update", "disable incoming transactions",
        "report the case to police", "review the potential fraud", "possible fraud",
        "request external fraud investigation", "2days", "receive result", "send reminder"
    ]

    baseline_prompt = f"""
You are given:
1. A list of 24 standard phrases used to describe a process model.
2. A text file describing transitions in a real process.

### Standard Transition Phrases:
{standard_transitions}

### Text File:
{text_file_content}

### Task:
- Carefully analyze the process described in the text.
- Try to map the text transitions to the 24 standard ones (be flexible, exact match is not required).
- After transition name "receive result", it must be a transition name with the label of "AND-split".   
- And this transition name with the label of "AND-split" must include transition names "enable customer profile update" and "enable incoming transactions" together.
- If this transition name with the label of "AND-split" does not include transition names "enable customer profile update" and "enable incoming transactions" together, return false and print "Please note that the process model shall enable customer profile update and incoming transaction in parallel." Otherwise, return true.
- You should Return one word: `true` or `false`. No need to print thr code.
"""
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "user", "content": baseline_prompt}],
    )

    return response.choices[0].message.content.strip()


if __name__ == "__main__":
    current_dir = os.getcwd()
    txt_filename = r"Insert the file path"

    txt_path = os.path.join(current_dir, txt_filename)

    txt_content = read_file_content(txt_path)

    # Baseline Evaluation
    print("\n⚙️ Running baseline evaluation (no DMN guidance)...\n")
    baseline_result = baseline_evaluate_text(txt_content)

    # Final result
    print("\n✅ Baseline Evaluation Result:\n", baseline_result)



## Rule5

In [ ]:
import os
from openai import OpenAI


# Get API key from environment variable
client = OpenAI(api_key="Insert API key") # Replace with your full API key

def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def baseline_evaluate_text(text_file_content):
    standard_transitions = [
        "registration manually", "registration online", "receive customer registration", "register request",
        "send credit request to UC", "receive result", "not ok", "ok", "send rejection letter", "End",
        "enbale customer profile update", "enable incoming transactions", "potential fraud using ML",
        "inform customer", "enable full access", "disable customer profile update", "disable incoming transactions",
        "report the case to police", "review the potential fraud", "possible fraud",
        "request external fraud investigation", "2days", "receive result", "send reminder"
    ]


    baseline_prompt = f"""
You are given:
1. A list of 24 standard phrases used to describe a process model.
2. A text file describing transitions in a real process.

### Standard Transition Phrases:
{standard_transitions}

### Text File:
{text_file_content}

### Task:
- Carefully analyze the process described in the text.
- Try to map the text transitions to the 24 standard ones (be flexible, exact match is not required).
- In the ProcessTransitions, after transition name "receive result", it must be a transition name with the label of "AND-split".   
- And this transition name with the label of "AND-split" must include transition names "enable customer profile update", "enable incoming transactions" and "potential fraud using ML" together.
- If this transition name with the label of "AND-split" must include transition names "enable customer profile update","enable incoming transactions" and "potential fraud using ML" together, return true.
- If this transition name with the label of "AND-split" does not include transition names "enable customer profile update", "enable incoming transactions" and "potential fraud using ML" together, return false and print "Please note that we can perform checking fraud using ML model earlie, as it will be automated and will not needed to be performed in the same position that manual fraud checking had been performed."
- You should Return one word: `true` or `false`. No need to print thr code.
"""
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "user", "content": baseline_prompt}],
    )

    return response.choices[0].message.content.strip()


if __name__ == "__main__":
    current_dir = os.getcwd()
    txt_filename = r"Insert the file path"

    txt_path = os.path.join(current_dir, txt_filename)

    txt_content = read_file_content(txt_path)

    # Baseline Evaluation
    print("\n⚙️ Running baseline evaluation (no DMN guidance)...\n")
    baseline_result = baseline_evaluate_text(txt_content)

    # Final result
    print("\n✅ Baseline Evaluation Result:\n", baseline_result)



## Rule6

In [ ]:
import os
from openai import OpenAI


# Get API key from environment variable
client = OpenAI(api_key="Insert API key") # Replace with your full API key


def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def baseline_evaluate_text(text_file_content):
    standard_transitions = [
        "registration manually", "registration online", "receive customer registration", "register request",
        "send credit request to UC", "receive result", "not ok", "ok", "send rejection letter", "End",
        "enbale customer profile update", "enable incoming transactions", "potential fraud using ML",
        "inform customer", "enable full access", "disable customer profile update", "disable incoming transactions",
        "report the case to police", "review the potential fraud", "possible fraud",
        "request external fraud investigation", "2days", "receive result", "send reminder"
    ]

    baseline_prompt = f"""
You are given:
1. A list of 24 standard phrases used to describe a process model.
2. A text file describing transitions in a real process.

### Standard Transition Phrases:
{standard_transitions}

### Text File:
{text_file_content}

### Task:
- Carefully analyze the process described in the text.
- Try to map the text transitions to the 24 standard ones (be flexible, exact match is not required).
- In the ProcessTransitions, there must be two transition names "2 days" and "receive result".   
- If there are two transition names "2 days" and "receive result" in the ProcessTransitions, it must be a same transition before them.
- If there are two transition names "2 days" and "receive result" in the ProcessTransitions, and there is a same transition before them, return true.
- If there are not two transition names "2 days" and "receive result" in the ProcessTransitions, and there is not a same transition before them, return false and print "Please note that we do not know if we will receive the result for external fraud investigation in 2 days in advanced".
- You should Return one word: `true` or `false`. No need to print thr code.
"""
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "user", "content": baseline_prompt}],
    )

    return response.choices[0].message.content.strip()


if __name__ == "__main__":
    current_dir = os.getcwd()
    txt_filename = r"Insert the file path"

    txt_path = os.path.join(current_dir, txt_filename)

    txt_content = read_file_content(txt_path)

    # Baseline Evaluation
    print("\n⚙️ Running baseline evaluation (no DMN guidance)...\n")
    baseline_result = baseline_evaluate_text(txt_content)

    # Final result
    print("\n✅ Baseline Evaluation Result:\n", baseline_result)


## Rule7

In [ ]:
import os
from openai import OpenAI


# Get API key from environment variable
client = OpenAI(api_key="Insert API key") # Replace with your full API key


def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def baseline_evaluate_text(text_file_content):
    standard_transitions = [
        "registration manually", "registration online", "receive customer registration", "register request",
        "send credit request to UC", "receive result", "not ok", "ok", "send rejection letter", "End",
        "enbale customer profile update", "enable incoming transactions", "potential fraud using ML",
        "inform customer", "enable full access", "disable customer profile update", "disable incoming transactions",
        "report the case to police", "review the potential fraud", "possible fraud",
        "request external fraud investigation", "2days", "receive result", "send reminder"
    ]

    baseline_prompt = f"""
You are given:
1. A list of 24 standard phrases used to describe a process model.
2. A text file describing transitions in a real process.

### Standard Transition Phrases:
{standard_transitions}

### Text File:
{text_file_content}

### Task:
- Carefully analyze the process described in the text.
- Try to map the text transitions to the 24 standard ones (be flexible, exact match is not required).
- In the ProcessTransitions, there must be two transition names "disable customer profile update" and "disable incoming transactions".   
- And it must be a same transition befor both transition names "disable customer profile update" and "disable incoming transactions".   
- If there are two transition names "disable customer profile update" and "disable incoming transactions" in the ProcessTransitions, and there is a same transition before them, return true.
- If there are not two transition names "2 days" and "receive result" in the ProcessTransitions, and there is not a same transition before them, return false and print  "Please note that disable customer profile update and disable incoming transactions shall be performed in parallel after identifying fraud."
- You should Return one word: `true` or `false`. No need to print thr code.
"""
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "user", "content": baseline_prompt}],
    )

    return response.choices[0].message.content.strip()


if __name__ == "__main__":
    current_dir = os.getcwd()
    txt_filename = r"Insert the file path"

    txt_path = os.path.join(current_dir, txt_filename)

    txt_content = read_file_content(txt_path)

    # Baseline Evaluation
    print("\n⚙️ Running baseline evaluation (no DMN guidance)...\n")
    baseline_result = baseline_evaluate_text(txt_content)

    # Final result
    print("\n✅ Baseline Evaluation Result:\n", baseline_result)

## Rule8

In [ ]:
import os
from openai import OpenAI


# Get API key from environment variable
client = OpenAI(api_key="Insert API key") # Replace with your full API key

def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def baseline_evaluate_text(text_file_content):
    standard_transitions = [
        "registration manually", "registration online", "receive customer registration", "register request",
        "send credit request to UC", "receive result", "not ok", "ok", "send rejection letter", "End",
        "enbale customer profile update", "enable incoming transactions", "potential fraud using ML",
        "inform customer", "enable full access", "disable customer profile update", "disable incoming transactions",
        "report the case to police", "review the potential fraud", "possible fraud",
        "request external fraud investigation", "2days", "receive result", "send reminder"
    ]

    baseline_prompt = f"""
You are given:
1. A list of 24 standard phrases used to describe a process model.
2. A text file describing transitions in a real process.

### Standard Transition Phrases:
{standard_transitions}

### Text File:
{text_file_content}

### Task:
- Carefully analyze the process described in the text.
- Try to map the text transitions to the 24 standard ones (be flexible, exact match is not required).
- In the ProcessTransitions, there must be two transition names  "send rejection letter" and "report".   
- And it must be a same transition befor these two transition names.
- If there are two transition names "send rejection letter" and "report" in the ProcessTransitions, and there is a same transition before them, return true.
- If there are not two transition names "send rejection letter" and "report" in the ProcessTransitions, and there is not a same transition before them, return false and print  "Please note that disable customer profile update and disable incoming transactions shall be performed in parallel after identifying fraud."
- You should Return one word: `true` or `false`. No need to print thr code.
"""
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "user", "content": baseline_prompt}],
    )

    return response.choices[0].message.content.strip()


if __name__ == "__main__":
    current_dir = os.getcwd()
    txt_filename = r"Insert the file path"

    txt_path = os.path.join(current_dir, txt_filename)

    txt_content = read_file_content(txt_path)

    # Baseline Evaluation
    print("\n⚙️ Running baseline evaluation (no DMN guidance)...\n")
    baseline_result = baseline_evaluate_text(txt_content)

    # Final result
    print("\n✅ Baseline Evaluation Result:\n", baseline_result)


## Rule9

In [ ]:
import os
from openai import OpenAI


# Get API key from environment variable
client = OpenAI(api_key="Insert API key") # Replace with your full API key


def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


def baseline_evaluate_text(text_file_content):
    standard_transitions = [
        "registration manually", "registration online", "receive customer registration", "register request",
        "send credit request to UC", "receive result", "not ok", "ok", "send rejection letter", "End",
        "enbale customer profile update", "enable incoming transactions", "potential fraud using ML",
        "inform customer", "enable full access", "disable customer profile update", "disable incoming transactions",
        "report the case to police", "review the potential fraud", "possible fraud",
        "request external fraud investigation", "2days", "receive result", "send reminder"
    ]

    baseline_prompt = f"""
You are given:
1. A list of 24 standard phrases used to describe a process model.
2. A text file describing transitions in a real process.

### Standard Transition Phrases:
{standard_transitions}

### Text File:
{text_file_content}

### Task:
- Carefully analyze the process described in the text.
- Try to map the text transitions to the 24 standard ones (be flexible, exact match is not required).
- In the ProcessTransitions, there must be two transition names "disable customer profile update" and "disable incoming transactions".   
- And also in addition, there must be two other transition names  "send rejection letter" and "report".   
- And it must be a same transition after two transition names "disable customer profile update" and "disable incoming transactions". 
- Also it must be a same transition before two transition names "send rejection letter" and "report". 
- Then the transition name after "disable customer profile update" and "disable incoming transactions" must be the transition name before "send rejection letter" and "report". 
- If the transition name after "disable customer profile update" and "disable incoming transactions" is the transition name before "send rejection letter" and "report" , return true.
- If the transition name after "disable customer profile update" and "disable incoming transactions" is not the transition name before "send rejection letter" and "report", return false and print  "Please note that reporting the case to police and send rejection letter shall be performed after disabling the services."
- You should Return one word: `true` or `false`. No need to print thr code.
"""
    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "user", "content": baseline_prompt}],
    )

    return response.choices[0].message.content.strip()


if __name__ == "__main__":
    current_dir = os.getcwd()
    txt_filename = r"Insert the file path"

    txt_path = os.path.join(current_dir, txt_filename)

    txt_content = read_file_content(txt_path)

    # Baseline Evaluation
    print("\n⚙️ Running baseline evaluation (no DMN guidance)...\n")
    baseline_result = baseline_evaluate_text(txt_content)

    # Final result
    print("\n✅ Baseline Evaluation Result:\n", baseline_result)
